In [1]:
import numpy as np
import pandas as pd
import scanpy as sc

## data preprocess

### 1.facs_Cd8

In [23]:
# adata = sc.read("/volume1/home/kliu/Others/cellTypeAbundance/data/PBMC_FACS/test_adata.h5ad")
adata_ref_pre = sc.read("/volume1/home/mhuang/cellTypeAbundance/data/FACS_CD8/train_adata_1.h5ad")
adata_test_pre = sc.read("/volume1/home/mhuang/cellTypeAbundance/data/FACS_CD8/test_adata_1.h5ad")
adata = adata_test_pre.concatenate(adata_ref_pre, batch_key="dataset_type")

/usr/local/anaconda3/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/tmp/ipykernel_4072521/3754856107.py:4: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata = adata_test_pre.concatenate(adata_ref_pre, batch_key="dataset_type")


In [41]:
# preprocess
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

sc.pp.normalize_total(adata,target_sum=1e4)
sc.pp.log1p(adata)
# sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
# adata = adata[:, adata.var.highly_variable]

sc.pp.scale(adata)

adata.var_names_make_unique()
adata.obs_names_make_unique()

In [42]:
adata_ref = adata[adata.obs['dataset_type']=='1']
adata_test = adata[adata.obs['dataset_type']=='0']

In [44]:
## 1.find marker genes
sc.tl.rank_genes_groups(adata_ref,groupby='cell.type',use_raw=False,method="wilcoxon") #method='t-test','wilcoxon'
# sc.pl.rank_genes_groups_matrixplot(adata,groupby='Pred_detail',values_to_plot='logfoldchanges',cmap='bwr')

## 2.select top_n marker genes
sc.tl.filter_rank_genes_groups(adata_ref,groupby="cell.type",min_in_group_fraction=0.8,max_out_group_fraction=0.2) 
# sc.pl.rank_genes_groups_dotplot(adata, groupby="Pred_detail", standard_scale="var", n_genes=top_n_genes,cmap='bwr')

/usr/local/anaconda3/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/usr/local/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/anaconda3/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:422: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/usr/local/anaconda3/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:422: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/usr/local/anaconda3/lib/python3.9/site-packages/scan

In [49]:
markers=sc.get.rank_genes_groups_df(adata_ref, group=adata_ref.obs['cell.type'].unique())
markers=markers.loc[~ markers.names.isna()]
markers['abs_score']=markers.scores.abs()
markers.sort_values('abs_score',ascending=False,inplace=True)
markers = markers.groupby('group').head(1500).sort_values('group')['names'].unique()#用list来画图

/tmp/ipykernel_4153746/3410864313.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  markers = markers.groupby('group').head(1500).sort_values('group')['names'].unique()#用list来画图


In [50]:
len(markers)

3440

In [51]:
adata_ref = adata_ref[:,markers]
adata_test = adata_test[:,markers]
adata_ref.obs.rename(columns={'cellType': 'cell.type'}, inplace=True)
adata_test.obs.rename(columns={'cellType': 'cell.type'}, inplace=True)

In [74]:
# adata_ref.obs['cell.type'] = adata_ref.obs['cell.type'].astype(str)
# adata_test.obs['cell.type'] = adata_test.obs['cell.type'].astype(str)

# adata_ref.write(f'/volume1/home/mhuang/cellTypeAbundance/data/FACS_CD8/FACS_CD8_train.h5ad')
adata_test.write(f'/volume1/home/mhuang/cellTypeAbundance/data/FACS_CD8/FACS_CD8_test.h5ad')

In [76]:
adata_test.obs['cell.type']

G_ACTAAAACCTTCTA-1-0    Naive Cytotoxic T cells
G_CATTTGACTCAGAC-1-0    Naive Cytotoxic T cells
G_TTACGTACCTGCTC-1-0    Naive Cytotoxic T cells
G_CTACCTCTCGACTA-1-0    Naive Cytotoxic T cells
G_ACGACCCTCATCAG-1-0    Naive Cytotoxic T cells
                                 ...           
H_TACTCCCTCATTGG-1-0              Naive T cells
H_AAGCCATGTTCGCC-1-0              Naive T cells
H_TAGCATCTTGACTG-1-0              Naive T cells
H_GCGGGACTAAGATG-1-0              Naive T cells
H_GACGCCGACCTGAA-1-0              Naive T cells
Name: cell.type, Length: 3999, dtype: category
Categories (9, object): ['B cells', 'CD4 T cells', 'CD14+ Monocytes', 'CD56 NK cells', ..., 'Memory T cells', 'Naive Cytotoxic T cells', 'Naive T cells', 'Regulatory T cells']

### 2.pbmc_facs

In [52]:
#pbmc_Facs
adata_a = sc.read("/volume3/kxu/scdata/PBMC_zheng/PBMCA.h5ad")
adata_b = sc.read("/volume3/kxu/scdata/PBMC_zheng/PBMCB.h5ad")
adata_c = sc.read("/volume3/kxu/scdata/PBMC_zheng/PBMCC.h5ad")
adata_ref = adata_a.concatenate([adata_b,adata_c], batch_key="batch")
#kn的数据是已经筛选过了的

#facs_cd8
adata_test = sc.read("/volume1/home/mhuang/cellTypeAbundance/data/FACS_CD8/test_adata_1.h5ad")

adata = adata_ref.concatenate(adata_test, batch_key="dataset_type")

/usr/local/anaconda3/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/tmp/ipykernel_4072521/1296860975.py:5: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata_ref = adata_a.concatenate([adata_b,adata_c], batch_key="batch")
/usr/local/anaconda3/lib/python3.9/site-

In [26]:
# preprocess
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

sc.pp.normalize_total(adata,target_sum=1e4)
sc.pp.log1p(adata)
# sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
# adata = adata[:, adata.var.highly_variable]

sc.pp.scale(adata)

adata.var_names_make_unique()
adata.obs_names_make_unique()

adata_ref = adata[adata.obs['dataset_type']=='0']
adata_test = adata[adata.obs['dataset_type']=='1']

sc.tl.rank_genes_groups(adata_ref,groupby='cell.type',use_raw=False,method="wilcoxon") #method='t-test','wilcoxon'
sc.tl.filter_rank_genes_groups(adata_ref,groupby="cell.type",min_in_group_fraction=0.8,max_out_group_fraction=0.2) 

markers=sc.get.rank_genes_groups_df(adata_ref, group=adata_ref.obs['cell.type'].unique())
markers=markers.loc[~ markers.names.isna()]
markers['abs_score']=markers.scores.abs()
markers.sort_values('abs_score',ascending=False,inplace=True)

/usr/local/anaconda3/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:582: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
/usr/local/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/usr/local/anaconda3/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:422: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/usr/local/anaconda3/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:422: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, 'logfoldchanges'] = np.log2(
/usr/local/anaconda3/lib/python3.9/site-packages/scan

In [27]:
markers = markers.groupby('group').head(1500).sort_values('group')['names'].unique()
len(markers)

/tmp/ipykernel_4072521/2031888245.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  markers = markers.groupby('group').head(1500).sort_values('group')['names'].unique()


3390

In [28]:
adata_ref = adata_ref[:,markers]
adata_test = adata_test[:,markers]
adata_ref.obs.rename(columns={'cellType': 'cell.type'}, inplace=True)
adata_test.obs.rename(columns={'cellType': 'cell.type'}, inplace=True)

In [31]:
# adata_ref.obs['cell.type'] = adata_ref.obs['cell.type'].astype(str)
# adata_test.obs['cell.type'] = adata_test.obs['cell.type'].astype(str)

# adata_ref.write(f'/volume1/home/mhuang/cellTypeAbundance/data/cross_batch/PBMC_FACS_train.h5ad')
adata_test.write(f'/volume1/home/mhuang/cellTypeAbundance/data/cross_batch/FACS_CD8_test.h5ad')

In [ ]:
## 我突然发现自己写错了，上面的是PBMC_Batch而不是PBMC_FACS数据集；从而虽然她们的基因名完全相等，
## 但是她们的细胞类型对不上的

In [40]:
#pbmc_facs
adata_ref=sc.read("/volume1/home/kliu/Others/cellTypeAbundance/data/PBMC_FACS/process_train.h5ad")
#这个是已经预处理过的

#facs_cd8
adata_test = sc.read("/volume1/home/mhuang/cellTypeAbundance/data/FACS_CD8/test_adata_1.h5ad")

/usr/local/anaconda3/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [41]:
hvgs = adata_ref.var.index


In [42]:
adata_test=adata_test[:,hvgs]


In [43]:
# preprocess
# sc.pp.filter_cells(adata_test, min_genes=200)
# sc.pp.filter_genes(adata_test, min_cells=3)

sc.pp.normalize_total(adata_test,target_sum=1e4)
sc.pp.log1p(adata_test)
# sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
# adata = adata[:, adata.var.highly_variable]

sc.pp.scale(adata_test)

adata_test.var_names_make_unique()
adata_test.obs_names_make_unique()


/usr/local/anaconda3/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [44]:
adata_test

AnnData object with n_obs × n_vars = 4000 × 3000
    obs: 'barcode', 'cell.type', 'batch'
    var: 'gene_symbols', 'mean', 'std'
    uns: 'log1p'

In [45]:
adata_test.write(f'/volume1/home/mhuang/cellTypeAbundance/data/cross_batch/FACS_CD8_test.h5ad')


## run PropDEC

### 1.FACS_CD8
all = healthy ,  train:test = 0.7*all : 0.3*all

In [2]:
from propDEC_end2end import propDEC
import scanpy as sc
import matplotlib.pyplot as plt

In [3]:
# create dataset
pipeline = propDEC(input_size=3440, num_classes=9, metric="t-sne", mode="soft")#soft/hard/km

train_dataset = pipeline.create_dataset("FACS_CD8", "FACS_CD8_train.h5ad", mode="train")
test_dataset = pipeline.create_dataset("FACS_CD8", "FACS_CD8_test.h5ad", mode="test")

ref_adata = train_dataset.train_adata
test_adata = test_dataset.test_adata

/usr/local/anaconda3/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [4]:
scores, proportion, f1scores, precision, recall = pipeline(epochs=5)

==> Saving Checkpoints
==> Finish training !


==> Saving Checkpoints
Got 3716 / 3999 with accuracy' 92.92%
==> resample


/volume1/home/mhuang/cellTypeAbundance/src/test/utils.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell_num = int(counts[index])
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/utils.py:260: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['C_TTGAACCTTGGTGT-1-1-1', 'C_CTATTGACGCCATA-1-1-1', 'C_TAGTTAGATATGCG-1-1-1', 'C_AAGGTGCTGGA

==> Write the resample annotated data to cache
==> Write the reweighted data to cache


/usr/local/anaconda3/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


...

==> Loading Checkpoints


==> Saving Checkpoints
Got 3760 / 3999 with accuracy' 94.02%
==> resample


/volume1/home/mhuang/cellTypeAbundance/src/test/utils.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell_num = int(counts[index])
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/utils.py:260: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['C_TTGAACCTTGGTGT-1-1-1', 'C_CTATTGACGCCATA-1-1-1', 'C_TAGTTAGATATGCG-1-1-1', 'C_AAGGTGCTGGA

==> Write the resample annotated data to cache
==> Write the reweighted data to cache
==> Finish!


/volume1/home/mhuang/cellTypeAbundance/src/test/propDEC_end2end.py:195: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ground_truth = self.test_adata.obs.groupby('cell.type').count()/len(self.test_adata)


In [5]:
proportion

,ref,predict_1,predict_2,predict_3,predict_4,predict_5,Ground_truth
B cells,0.300237,0.2996,0.3000,0.2999,0.3004,0.2999,0.300075
CD4 T cells,0.020018,0.0143,0.0234,0.0155,0.0163,0.0151,0.020005
CD14+ Monocytes,0.100090,0.0992,0.0989,0.0990,0.0992,0.0988,0.100025
CD56 NK cells,0.020018,0.0202,0.0208,0.0204,0.0201,0.0209,0.020005
Cytotoxic T cells,0.100090,0.3574,0.3765,0.3795,0.3683,0.3701,0.400100
Memory T cells,0.020018,0.0164,0.0178,0.0183,0.0170,0.0179,0.020005
Naive Cytotoxic T cells,0.399560,0.1500,0.1269,0.1279,0.1357,0.1336,0.099775
Naive T cells,0.019985,0.0234,0.0180,0.0212,0.0239,0.0229,0.020005
Regulatory T cells,0.019985,0.0194,0.0177,0.0185,0.0190,0.0209,0.020005


In [6]:
f1scores

,predict_1,predict_2,predict_3,predict_4,predict_5
f1score_micro,0.929232,0.940235,0.939735,0.936484,0.940235
f1score_macro,0.837174,0.831750,0.825904,0.843094,0.845543


In [7]:
precision

,predict_1,predict_2,predict_3,predict_4,predict_5
B cells,0.999165,0.998332,0.998333,0.997502,0.999166
CD4 T cells,0.698113,0.537634,0.606557,0.696970,0.706897
CD14+ Monocytes,0.997481,1.000000,1.000000,0.994962,1.000000
CD56 NK cells,0.975309,0.951807,0.962963,0.987500,0.951807
Cytotoxic T cells,0.995819,0.989376,0.987434,0.992527,0.993252
Memory T cells,0.923077,0.915493,0.875000,0.897059,0.887324
Naive Cytotoxic T cells,0.668908,0.763314,0.752418,0.718750,0.734082
Naive T cells,0.636364,0.671233,0.616279,0.648936,0.681319
Regulatory T cells,0.750000,0.666667,0.693333,0.688312,0.674419


In [8]:
recall

,predict_1,predict_2,predict_3,predict_4,predict_5
B cells,0.997500,0.997500,0.998333,0.998333,0.998333
CD4 T cells,0.462500,0.625000,0.462500,0.575000,0.512500
CD14+ Monocytes,0.990000,0.987500,0.987500,0.987500,0.987500
CD56 NK cells,0.987500,0.987500,0.975000,0.987500,0.987500
Cytotoxic T cells,0.893125,0.931250,0.933125,0.913125,0.920000
Memory T cells,0.750000,0.812500,0.787500,0.762500,0.787500
Naive Cytotoxic T cells,0.997494,0.969925,0.974937,0.979950,0.982456
Naive T cells,0.787500,0.612500,0.662500,0.762500,0.775000
Regulatory T cells,0.712500,0.600000,0.650000,0.662500,0.725000


### 2.PBMC_FACS & FACS_CD8

train=pbmc_facs_all ,  test=facs_cd8_test

In [2]:
from propDEC_end2end import propDEC
import scanpy as sc
import matplotlib.pyplot as plt

In [3]:
# create dataset
pipeline = propDEC(input_size=3000, num_classes=9, metric="t-sne", mode="soft")#soft/hard/km

train_dataset = pipeline.create_dataset("cross_batch", "ref_adata.h5ad", mode="train")
test_dataset = pipeline.create_dataset("cross_batch", "test_adata.h5ad", mode="test")

ref_adata = train_dataset.train_adata
test_adata = test_dataset.test_adata


/usr/local/anaconda3/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [6]:
scores, proportion, f1scores, precision, recall = pipeline(epochs=5)


==> Saving Checkpoints
==> Finish training !


==> Saving Checkpoints
Got 6653 / 7491 with accuracy' 88.81%
==> resample
==> Write the resample annotated data to cache


/volume1/home/mhuang/cellTypeAbundance/src/test/utils.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell_num = int(counts[index])
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


==> Write the reweighted data to cache


/usr/local/anaconda3/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


...

==> Loading Checkpoints


==> Saving Checkpoints
Got 6748 / 7491 with accuracy' 90.08%
==> resample
==> Write the resample annotated data to cache


/volume1/home/mhuang/cellTypeAbundance/src/test/utils.py:90: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell_num = int(counts[index])
/usr/local/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


==> Write the reweighted data to cache
==> Finish!


/volume1/home/mhuang/cellTypeAbundance/src/test/propDEC_end2end.py:195: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ground_truth = self.test_adata.obs.groupby('cell.type').count()/len(self.test_adata)


In [7]:
proportion

,ref,predict_1,predict_2,predict_3,predict_4,predict_5,Ground_truth
B cells,0.104391,0.2981,0.2990,0.2989,0.2975,0.2991,0.300227
CD4 T cells,0.232187,0.0405,0.0411,0.0490,0.0476,0.0508,0.020024
CD14+ Monocytes,0.027030,0.0973,0.0984,0.0984,0.0985,0.0981,0.100120
CD56 NK cells,0.086785,0.0201,0.0198,0.0198,0.0204,0.0201,0.020024
Cytotoxic T cells,0.105634,0.1124,0.1032,0.1059,0.1043,0.1037,0.100120
Memory T cells,0.105841,0.0305,0.0192,0.0190,0.0182,0.0177,0.020024
Naive Cytotoxic T cells,0.123447,0.3541,0.3939,0.3877,0.3950,0.3916,0.399680
Naive T cells,0.108430,0.0278,0.0149,0.0116,0.0098,0.0090,0.019891
Regulatory T cells,0.106255,0.0192,0.0105,0.0097,0.0087,0.0101,0.019891


In [8]:
f1scores


,predict_1,predict_2,predict_3,predict_4,predict_5
f1score_micro,0.888132,0.906555,0.903084,0.902149,0.900814
f1score_macro,0.755765,0.770131,0.743303,0.734445,0.732707


In [9]:
precision


,predict_1,predict_2,predict_3,predict_4,predict_5
B cells,0.998210,0.998216,0.998660,0.999102,0.998214
CD4 T cells,0.374126,0.398693,0.333333,0.352778,0.333333
CD14+ Monocytes,0.993188,0.995946,0.997294,0.998643,0.998641
CD56 NK cells,1.000000,0.993289,1.000000,0.973684,0.986667
Cytotoxic T cells,0.696560,0.736074,0.742268,0.746094,0.746719
Memory T cells,0.513514,0.770270,0.741259,0.748148,0.751938
Naive Cytotoxic T cells,0.953488,0.928620,0.937884,0.929123,0.932226
Naive T cells,0.428571,0.588785,0.540230,0.594203,0.545455
Regulatory T cells,0.621429,0.746667,0.685714,0.621212,0.644737


In [10]:
recall

,predict_1,predict_2,predict_3,predict_4,predict_5
B cells,0.991996,0.995109,0.994220,0.989329,0.994220
CD4 T cells,0.713333,0.813333,0.800000,0.846667,0.846667
CD14+ Monocytes,0.972000,0.982667,0.982667,0.981333,0.980000
CD56 NK cells,0.986667,0.986667,0.980000,0.986667,0.986667
Cytotoxic T cells,0.756000,0.740000,0.768000,0.764000,0.758667
Memory T cells,0.760000,0.760000,0.706667,0.673333,0.646667
Naive Cytotoxic T cells,0.862725,0.921176,0.917836,0.923848,0.918838
Naive T cells,0.583893,0.422819,0.315436,0.275168,0.241611
Regulatory T cells,0.583893,0.375839,0.322148,0.275168,0.328859
